<a href="https://colab.research.google.com/github/HengNguan/cuda_learning/blob/google_colab/Cuda_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THIS IS JUST A SELF LEARN TRACKING
Reference: https://www.youtube.com/watch?v=86FAWCzIe_4&list=PLvUs2zO9Ewbpo_u1CnnZPx9-GdDK2LC-V

# Chapter 1: PoC, feel and see

### Check CUDA Env

In [1]:
!nvidia-smi
!nvcc --version

Tue Dec 16 05:43:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter


Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmppwid_cle".


### For pure CUDA kernel

In [8]:
%%cuda -c "-I /does/not/exist -arch sm_75"

/** Based on some reseach on intenet, this is required everytab as the
 nvidia-smi showed CUDA version is 12.4, BUT
 nvcc --version showed V12.5.82 **/

/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x1b): undefined reference to `main'
collect2: error: ld returned 1 exit status



In [9]:
%%cuda -c "-I /does/not/exist -arch sm_75"
#include <stdio.h>

// This is a special function that runs on the GPU (device) instead of the CPU (host)
__global__ void kernel() {
  printf("Hello world!\n");
}

int main() {
  // Invoke the kernel function on the GPU with one block of one thread
  kernel<<<1,1>>>();

  // Check for error codes (remember to do this for _every_ CUDA function)
  if(cudaDeviceSynchronize() != cudaSuccess) {
    fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(cudaPeekAtLastError()));
  }
  return 0;
}

Hello world!



In [10]:
%%cuda -c "-I /does/not/exist -arch sm_75"
#include <stdio.h>

__global__ void whoami(void) {
    int block_id =
        blockIdx.x +    // apartment number on this floor (points across)
        blockIdx.y * gridDim.x +    // floor number in this building (rows high)
        blockIdx.z * gridDim.x * gridDim.y;   // building number in this city (panes deep)

    int block_offset =
        block_id * // times our apartment number
        blockDim.x * blockDim.y * blockDim.z; // total threads per block (people per apartment)

    int thread_offset =
        threadIdx.x +
        threadIdx.y * blockDim.x +
        threadIdx.z * blockDim.x * blockDim.y;

    int id = block_offset + thread_offset; // global person id in the entire apartment complex

    printf("%04d | Block(%d %d %d) = %3d | Thread(%d %d %d) = %3d\n",
        id,
        blockIdx.x, blockIdx.y, blockIdx.z, block_id,
        threadIdx.x, threadIdx.y, threadIdx.z, thread_offset);
    // printf("blockIdx.x: %d, blockIdx.y: %d, blockIdx.z: %d, threadIdx.x: %d, threadIdx.y: %d, threadIdx.z: %d\n", blockIdx.x, blockIdx.y, blockIdx.z, threadIdx.x, threadIdx.y, threadIdx.z);
}

int main(int argc, char **argv) {
    const int b_x = 2, b_y = 3, b_z = 4;
    const int t_x = 4, t_y = 4, t_z = 4; // the max warp size is 32, so
    // we will get 2 warp of 32 threads per block

    int blocks_per_grid = b_x * b_y * b_z;
    int threads_per_block = t_x * t_y * t_z;

    printf("%d blocks/grid\n", blocks_per_grid);
    printf("%d threads/block\n", threads_per_block);
    printf("%d total threads\n", blocks_per_grid * threads_per_block);

    dim3 blocksPerGrid(b_x, b_y, b_z); // 3d cube of shape 2*3*4 = 24
    dim3 threadsPerBlock(t_x, t_y, t_z); // 3d cube of shape 4*4*4 = 64

    whoami<<<blocksPerGrid, threadsPerBlock>>>();
    cudaDeviceSynchronize();
}

24 blocks/grid
64 threads/block
1536 total threads
1088 | Block(1 2 2) =  17 | Thread(0 0 0) =   0
1089 | Block(1 2 2) =  17 | Thread(1 0 0) =   1
1090 | Block(1 2 2) =  17 | Thread(2 0 0) =   2
1091 | Block(1 2 2) =  17 | Thread(3 0 0) =   3
1092 | Block(1 2 2) =  17 | Thread(0 1 0) =   4
1093 | Block(1 2 2) =  17 | Thread(1 1 0) =   5
1094 | Block(1 2 2) =  17 | Thread(2 1 0) =   6
1095 | Block(1 2 2) =  17 | Thread(3 1 0) =   7
1096 | Block(1 2 2) =  17 | Thread(0 2 0) =   8
1097 | Block(1 2 2) =  17 | Thread(1 2 0) =   9
1098 | Block(1 2 2) =  17 | Thread(2 2 0) =  10
1099 | Block(1 2 2) =  17 | Thread(3 2 0) =  11
1100 | Block(1 2 2) =  17 | Thread(0 3 0) =  12
1101 | Block(1 2 2) =  17 | Thread(1 3 0) =  13
1102 | Block(1 2 2) =  17 | Thread(2 3 0) =  14
1103 | Block(1 2 2) =  17 | Thread(3 3 0) =  15
1104 | Block(1 2 2) =  17 | Thread(0 0 1) =  16
1105 | Block(1 2 2) =  17 | Thread(1 0 1) =  17
1106 | Block(1 2 2) =  17 | Thread(2 0 1) =  18
1107 | Block(1 2 2) =  17 | Thread(3 

### Make use of python library, to write GPU kernel in python

In [11]:
from numba import cuda

# CUDA kernel to add two arrays using Numba
@cuda.jit
def addKernel(a, b, c):
    tid = cuda.grid(1)
    if tid < a.shape[0]:
        c[tid] = a[tid] + b[tid]

In [12]:
import numpy as np
from numba import cuda

# Prepare host data
n = 1000000  # Size of arrays
a_host = np.random.randint(0, 100, n).astype(np.int32)
b_host = np.random.randint(0, 100, n).astype(np.int32)
c_host = np.zeros_like(a_host)

# Transfer data to device
a_device = cuda.to_device(a_host)
b_device = cuda.to_device(b_host)
c_device = cuda.to_device(c_host)

# Define grid and block dimensions
threads_per_block = 256
blocks_per_grid = (n + (threads_per_block - 1)) // threads_per_block

# Call the Numba CUDA kernel
addKernel[blocks_per_grid, threads_per_block](a_device, b_device, c_device)
cuda.synchronize() # Ensure kernel completes

# Copy result back to host
c_result_host = c_device.copy_to_host()

# Verify the result
host_c_expected = a_host + b_host

# Check for correctness
if np.array_equal(c_result_host, host_c_expected):
    print("CUDA kernel executed successfully and results match CPU computation!")
else:
    print("Error: CUDA kernel results do not match CPU computation.")

# Display a few elements for verification
print("\nFirst 5 elements of A:", a_host[:5])
print("First 5 elements of B:", b_host[:5])
print("First 5 elements of C (from CUDA):", c_result_host[:5])
print("First 5 elements of C (from CPU):", host_c_expected[:5])

CUDA kernel executed successfully and results match CPU computation!

First 5 elements of A: [85 73 18 29 17]
First 5 elements of B: [45 64  5 53 90]
First 5 elements of C (from CUDA): [130 137  23  82 107]
First 5 elements of C (from CPU): [130 137  23  82 107]


# Chapter 1.1: Vector Add and compare

In [18]:
# Start with the magic line <-- Still Python
%%cuda -c "-I /does/not/exist -arch sm_75"

// Start C++ / CU CODE

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>


#define N 10000000
#define BLOCK_SIZE 256

// CPU Vector addition
void vector_add_cpu(float *a, float *b, float *c, int n)
{
    for (int i = 0; i < n; i++)
    {
        c[i] = a[i] + b[i];
    }
}

// CUDA kernel for vector addition
__global__ void vector_add_gpu(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

// Initialize vector with random values
void init_vector(float *vec, int n) {
    for (int i = 0; i < n; i++) {
        vec[i] = (float)rand() / RAND_MAX;
    }
}

// Time Measuring
double get_time() {
    struct timespec ts;
    clock_gettime(CLOCK_MONOTONIC, &ts);
    return ts.tv_sec + ts.tv_nsec * 1e-9;
}


// Main function
int main() {

    float *h_a, *h_b, *h_c_cpu, *h_c_gpu;
    float *d_a, *d_b, *d_c;
    size_t size = N * sizeof(float);

    // Allocate host memory
    h_a = (float *)malloc(size);
    h_b = (float *)malloc(size);
    h_c_cpu = (float *)malloc(size);
    h_c_gpu = (float *)malloc(size);

    // printf("Hello World !");

    // Initialize vectors on host
    srand(time(NULL));
    init_vector(h_a, N);
    init_vector(h_b, N);

    // Allocate device memory
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data to device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    int num_blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    // N = 1024, BLOCK_SIZE = 256, num_blocks = 4
    // (N + BLOCK_SIZE - 1) / BLOCK_SIZE = ( (1025 + 256 - 1) / 256 ) = 1280 / 256 = 4 rounded

    printf("Performing warm-up runs...\n");
    for (int i = 0; i < 3; i++) {
        vector_add_cpu(h_a, h_b, h_c_cpu, N);
        vector_add_gpu<<<num_blocks, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
        cudaDeviceSynchronize();
    }

    printf("Benchmarking CPU implementation...\n");
    double cpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        vector_add_cpu(h_a, h_b, h_c_cpu, N);
        double end_time = get_time();
        cpu_total_time += end_time - start_time;
    }
    double cpu_avg_time = cpu_total_time / 20.0;

    printf("Benchmarking GPU implementation...\n");
    double gpu_total_time = 0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        vector_add_gpu<<<num_blocks, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
        cudaDeviceSynchronize();
        double end_time = get_time();
        gpu_total_time += end_time - start_time;
    }
    double gpu_avg_time = gpu_total_time / 20.0;


    printf("CPU average time: %f milliseconds\n", cpu_avg_time*1000);
    printf("GPU average time: %f milliseconds\n", gpu_avg_time*1000);
    printf("Speedup: %fx\n", cpu_avg_time / gpu_avg_time);


    free(h_a);
    free(h_b);
    free(h_c_cpu);
    free(h_c_gpu);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);


    return 0;
}


Performing warm-up runs...
Benchmarking CPU implementation...
Benchmarking GPU implementation...
CPU average time: 38.931868 milliseconds
GPU average time: 0.469699 milliseconds
Speedup: 82.886853x

